In [1]:
import pandas as pd
import pyomo.environ as pye
import pyomo.opt as pyo

![imageinfo](https://miro.medium.com/max/1280/1*haKSsgOaPd_oON5IlyAkIg.png)

## Plant Location  

#### Manufacturing variable costs

In [2]:
# Import Costs
manvar_costs = pd.read_excel(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\variable_costs.xlsx", index_col = 0)
manvar_costs

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


#### Freight costs

In [3]:
# Import Costs
freight_costs = pd.read_excel(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\freight_costs.xlsx", index_col = 0)
freight_costs

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,0,12250,1100,16100,8778
Germany,13335,0,8617,20244,10073
Japan,15400,22750,0,43610,14350
Brazil,16450,22050,28000,0,29750
India,13650,15400,24500,29400,0


#### Variable Costs

In [5]:
# Variable Costs
var_cost = freight_costs/1000 + manvar_costs 
var_cost

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


#### Fixed Costs

In [6]:
# Import Costs
fixed_costs = pd.read_excel(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\fixed_cost.xlsx", index_col = 0)
fixed_costs

,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


#### Plants Capacity

In [7]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\capacity.xlsx", index_col = 0)
cap

,Low,High
Capacity (kUnits/month),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


#### Demand 

In [8]:
# -- Demand
demand = pd.read_excel(r"D:\Downloads\Chrome Downloads\supply-chain-optimization-main\demand.xlsx", index_col = 0)
demand

,Demand
(Units/month),
USA,2800000
Germany,90000
Japan,1700000
Brazil,145000
India,160000


![image info](https://miro.medium.com/max/1400/1*rtP7otnvgY2nT-ONqtAM6A.png)

# Creating Lists and Dicts

In [9]:
locations = ["USA", "Germany", "Japan", "Brazil", "India"]
size = ["Low", "High"]

In [10]:
demand_dic = dict({i:demand.loc[i,"Demand"] for i in locations})
demand_dic

{'USA': 2800000,
 'Germany': 90000,
 'Japan': 1700000,
 'Brazil': 145000,
 'India': 160000}

In [11]:
prod_vcost = dict(var_cost.stack())
prod_vcost

{('USA', 'USA'): 12.0,
 ('USA', 'Germany'): 24.25,
 ('USA', 'Japan'): 13.1,
 ('USA', 'Brazil'): 28.1,
 ('USA', 'India'): 20.778,
 ('Germany', 'USA'): 26.335,
 ('Germany', 'Germany'): 13.0,
 ('Germany', 'Japan'): 21.617,
 ('Germany', 'Brazil'): 33.244,
 ('Germany', 'India'): 23.073,
 ('Japan', 'USA'): 25.4,
 ('Japan', 'Germany'): 32.75,
 ('Japan', 'Japan'): 10.0,
 ('Japan', 'Brazil'): 53.61,
 ('Japan', 'India'): 24.35,
 ('Brazil', 'USA'): 24.45,
 ('Brazil', 'Germany'): 30.05,
 ('Brazil', 'Japan'): 36.0,
 ('Brazil', 'Brazil'): 8.0,
 ('Brazil', 'India'): 37.75,
 ('India', 'USA'): 18.65,
 ('India', 'Germany'): 20.4,
 ('India', 'Japan'): 29.5,
 ('India', 'Brazil'): 34.4,
 ('India', 'India'): 5.0}

In [12]:
prod_size = dict(fixed_costs.stack()*1000)
prod_size

{('USA', 'Low'): 6500000,
 ('USA', 'High'): 9500000,
 ('Germany', 'Low'): 4980000,
 ('Germany', 'High'): 7270000,
 ('Japan', 'Low'): 6230000,
 ('Japan', 'High'): 9100000,
 ('Brazil', 'Low'): 3230000,
 ('Brazil', 'High'): 4730000,
 ('India', 'Low'): 2110000,
 ('India', 'High'): 6160000}

In [13]:
cap_dic = dict(cap.stack()*1000)
cap_dic

{('USA', 'Low'): 500000,
 ('USA', 'High'): 1500000,
 ('Germany', 'Low'): 500000,
 ('Germany', 'High'): 1500000,
 ('Japan', 'Low'): 500000,
 ('Japan', 'High'): 1500000,
 ('Brazil', 'Low'): 500000,
 ('Brazil', 'High'): 1500000,
 ('India', 'Low'): 500000,
 ('India', 'High'): 3000000}

# Model Creation

In [14]:
model = pye.ConcreteModel()

### Lists as Sets

In [15]:
model.locations = pye.Set(initialize = locations)
model.size = pye.Set(initialize = size)

### Dict as Param

In [16]:
model.prod_vcost = pye.Param(model.locations, model.locations, initialize = prod_vcost)
model.prod_fcost = pye.Param(model.locations, model.size, initialize = prod_size)
model.cap_h_l = pye.Param(model.locations, model.size, initialize = cap_dic)
model.dem = pye.Param(model.locations, initialize = demand_dic)

### Desicion making Variables as Variables

In [17]:
model.prod_cap = pye.Var(model.locations, model.size, domain = pye.Binary)
model.x = pye.Var(model.locations, model.locations, domain = pye.NonNegativeReals)

### Contraints

In [18]:
def con_satisfaction(model, j):
    return sum(model.x[i, j] for i in model.locations) >= model.dem[j]
model.con_satisfaction = pye.Constraint(model.locations, rule=con_satisfaction)

In [19]:
def con_transportation(model, i):
    return sum(model.x[i,j] for j in model.locations) <= sum(model.cap_h_l[i,j] * model.prod_cap[i,j] for j in model.size)
model.con_transportation = pye.Constraint(model.locations, rule=con_transportation)

### Objective - Mixed Interger equation

# Capacitated Facility Location
To demonstrate blocks, let's setup and solve an instance of the CFL model.

Sets:
- $I$: supply sites, indexed by $i$
- $J$: demand sites, indexed by $j$

Parameters:
- $s_i$: supply capacity of supply site $i$
- $d_j$: demand required by demand site $j$
- $f_i$: fixed cost to open supply site $i$
- $c_{ij}$: variable cost to transport from supply site $i$ to demand site $j$

Variables:
- $x_{ij}$ - quantity of product to ship from supply site $i$ to demand site $j$
- $y_i$ - 0/1 decision variable indicating that supply site $i$ is producing

Model:
$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{subject to  }
& \sum_{i \in I} x_{ij} \ge d_j && \forall j \in J \\
& \sum_{j \in J} x_{ij} \le s_i y_i && \forall i \in I \\
& x \in \mathbb{R}_+^{|I| \times |J|} && \\
& y \in \{0, 1\}^{|I|} && \\
\end{alignat*}
$$

In [20]:
expr = sum(model.prod_fcost[i,j] * model.prod_cap[i,j] for i in model.locations for j in model.size) + sum(model.prod_vcost[i,j]*model.x[i,j] for i in model.locations for j in model.locations)
model.objective = pye.Objective(expr = expr)

### Selecting Solver and optimization

In [21]:
solver = pyo.SolverFactory("glpk")
results = solver.solve(model,tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\rshib\AppData\Local\Temp\tmp4hzziiq2.glpk.raw --wglp C:\Users\rshib\AppData\Local\Temp\tmpdxm9zlds.glpk.glp
 --cpxlp C:\Users\rshib\AppData\Local\Temp\tmpara__sj2.pyomo.lp
Reading problem data from 'C:\Users\rshib\AppData\Local\Temp\tmpara__sj2.pyomo.lp'...
C:\Users\rshib\AppData\Local\Temp\tmpara__sj2.pyomo.lp:173: warning: lower bound of variable 'x1' redefined
C:\Users\rshib\AppData\Local\Temp\tmpara__sj2.pyomo.lp:173: warning: upper bound of variable 'x1' redefined
11 rows, 36 columns, 61 non-zeros
10 integer variables, all of which are binary
183 lines were read
Writing problem data to 'C:\Users\rshib\AppData\Local\Temp\tmpdxm9zlds.glpk.glp'...
183 lines were written
GLPK Integer Optimizer 5.0
11 rows, 36 columns, 61 non-zeros
10 integer variables, all of which are binary
Preprocessing...
10 rows, 35 columns, 60 non-zeros
10 integer variables, all of which are binary
Scaling...
 A: min|aij

# Optimal quantity to produce and supply to minimize cost

In [22]:
model.display()

Model unknown

  Variables:
    prod_cap : Size=10, Index=prod_cap_index
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
         ('Brazil', 'High') :     0 :   0.0 :     1 : False : False : Binary
          ('Brazil', 'Low') :     0 :   1.0 :     1 : False : False : Binary
        ('Germany', 'High') :     0 :   0.0 :     1 : False : False : Binary
         ('Germany', 'Low') :     0 :   0.0 :     1 : False : False : Binary
          ('India', 'High') :     0 :   1.0 :     1 : False : False : Binary
           ('India', 'Low') :     0 :   0.0 :     1 : False : False : Binary
          ('Japan', 'High') :     0 :   1.0 :     1 : False : False : Binary
           ('Japan', 'Low') :     0 :   0.0 :     1 : False : False : Binary
            ('USA', 'High') :     0 :   1.0 :     1 : False : False : Binary
             ('USA', 'Low') :     0 :   0.0 :     1 : False : False : Binary
    x : Size=25, Index=x_index
        Key                    : Lower : Value   

### No. of objectives, constraints & variables

In [23]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 92981000.0
  Upper bound: 92981000.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 36
  Number of nonzeros: 61
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 15
      Number of created subproblems: 15
  Error rc: 0
  Time: 0.09254646301269531
# ----------------------------------------------------------
#   Solution Information
# ----------------------